https://www.kaggle.com/code/faressayah/lending-club-loan-defaulters-prediction

# 💾 Data Description

- loan_amnt             
  - L'importo indicato del prestito richiesto dal richiedente. Se in un certo momento il dipartimento crediti riduce l'importo del prestito, ciò sarà riflesso in questo valore.
- term                  
  - Il numero di pagamenti del prestito. I valori sono in mesi e possono essere sia 36 che 60.
- int_rate              
  - Tasso di interesse sul prestito
- installment           
  - Il pagamento mensile dovuto dal richiedente se il prestito è erogato.
- grade                 
  - Grado di prestito assegnato da LC
- sub_grade             
  - Sotto-grado di prestito assegnato da LC
- emp_title             
  - Il titolo di lavoro fornito dal richiedente al momento della richiesta del prestito.
- emp_length            
  - Durata dell'occupazione in anni. I valori possibili sono compresi tra 0 e 10 dove 0 significa meno di un anno e 10 significa dieci o più anni.
- home_ownership        
  - Lo stato di proprietà della casa fornito dal richiedente durante la registrazione o ottenuto dal rapporto di credito. I nostri valori sono: AFFITTO, PROPRIETARIO, MUTUO IPOTECARIO, ALTRO
- annual_inc            
  - Il reddito annuale autodichiarato fornito dal richiedente durante la registrazione.
- verification_status   
  - Indica se il reddito è stato verificato da LC, non verificato o se la fonte del reddito è stata verificata
- issue_d               
  - Il mese in cui il prestito è stato finanziato
- loan_status           
  - Stato attuale del prestito
- purpose               
  - Una categoria fornita dal richiedente per la richiesta di prestito.
- title                 
  - Il titolo del prestito fornito dal richiedente
- zip_code              
  - I primi 3 numeri del codice postale forniti dal richiedente nella domanda di prestito.
- addr_state            
  - Lo stato fornito dal richiedente nella domanda di prestito
- dti                   
  - Un rapporto calcolato utilizzando i pagamenti mensili totali del debito del richiedente sul totale degli obblighi di debito, escludendo il mutuo e il prestito LC richiesto, diviso per il reddito mensile autodichiarato del richiedente.
- earliest_cr_line      
  - Il mese in cui è stata aperta la prima linea di credito segnalata del richiedente
- open_acc              
  - Il numero di linee di credito aperte nel file di credito del richiedente.
- pub_rec               
  - Numero di record pubblici negativi
- revol_bal             
  - Bilancio totale del credito rotativo
- revol_util            
  - Tasso di utilizzo della linea di credito rotativo, o l'importo di credito utilizzato dal richiedente rispetto al totale del credito rotativo disponibile.
- total_acc             
  - Il numero totale di linee di credito attualmente nel file di credito del richiedente
- initial_list_status   
  - Lo stato iniziale dell'elenco del prestito. I valori possibili sono - W, F
- application_type      
  - Indica se il prestito è una domanda individuale o una domanda congiunta con due co-richiedenti
- mort_acc              
  - Numero di conti ipotecari.
- pub_rec_bankruptcies  
  - Numero di fallimenti registrati pubblicamente


In [24]:
!pip install -q hvplot

In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats 
import matplotlib.pyplot as plt
import hvplot.pandas

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report, 
    roc_auc_score, roc_curve, auc
)
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

pd.set_option('display.float', '{:.2f}'.format)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [26]:
data = pd.read_csv("C:/Users/edoardo.frapiccini/Desktop/vai/AI/pratica/Machine learning for credit scoring/progetti/Lending Club Loan Defaulters Prediction/dataset/Lending Club Dataset/lending_club_loan_two.csv")

#elimino tutti i valori mancanti
data.dropna()

#elimino le colonne che non mi servono
data.drop('emp_length', axis=1, inplace=True)
data.drop('title', axis=1, inplace=True)
data.drop('grade', axis=1, inplace=True)
data.drop('issue_d', axis=1, inplace=True)

#converto i valori categorici in numerici
""" data.drop('emp_title', axis=1, inplace=True) #troppe categorie diverse """
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['emp_title'] = le.fit_transform(data['emp_title'])

term_values = {' 36 months': 36, ' 60 months': 60}
data['term'] = data.term.map(term_values)

data['earliest_cr_line'] = pd.to_datetime(data['earliest_cr_line'])
data['earliest_cr_line'] = data.earliest_cr_line.dt.year

data['zip_code'] = data.address.apply(lambda x: x[-5:])
data = pd.get_dummies(data, columns=['zip_code'], drop_first=True)
data.drop('address', axis=1, inplace=True)

dummies = ['loan_status', 'sub_grade', 'verification_status', 'purpose', 'initial_list_status', 
           'application_type', 'home_ownership']
data = pd.get_dummies(data, columns=dummies, drop_first=True)

#converto i valori booleani in numerici
bool_columns = data.select_dtypes(include='bool').columns
data[bool_columns] = data[bool_columns].astype(int)

#-------
# Calcola la correlazione rispetto alla colonna 'loan_status_Fully Paid'
correlation = data.corr()['loan_status_Fully Paid'].drop('loan_status_Fully Paid')

# Trova le colonne con correlazione compresa tra 0.049 e -0.049
columns_to_drop = correlation.loc[((correlation > -0.049) & (correlation < 0.049)) & (correlation.index != 'installment')].index

# Elimina le colonne dal DataFrame
data.drop(columns=columns_to_drop, inplace=True)

data = data.dropna()

data_1 = data[data['loan_status_Fully Paid'] == 1]
data_0 = data[data['loan_status_Fully Paid'] == 0]

train, test = train_test_split(data_1, test_size=0.33, random_state=42)

#rimuovo i valori anomali
train = train[train['annual_inc'] <= 250000]
train = train[train['dti'] <= 50]

X_train, y_train = train.drop('installment', axis=1), train.installment
X_test, y_test = test.drop('installment', axis=1), test.installment

""" scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)  """

X_train = np.array(X_train).astype(np.float32)
X_test = np.array(X_test).astype(np.float32)
y_train = np.array(y_train).astype(np.float32)
y_test = np.array(y_test).astype(np.float32)

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.linear_model import Lasso

# Definisco il modello con regressione polinomiale e penalizzazione Lasso
model = Pipeline([
    ('poly_features', PolynomialFeatures(degree=2)),
    ('lasso', Lasso(alpha=1))
])

# Addestramento del modello
model.fit(X_train, y_train)
# Predizione
y_test_pred = model.predict(X_test)
results = pd.DataFrame({'True Values': y_test, 'Predicted Values': y_test_pred})
print(results.head(10))

C:\Users\edoardo.frapiccini\AppData\Local\Temp\ipykernel_8880\3633823342.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['earliest_cr_line'] = pd.to_datetime(data['earliest_cr_line'])


   True Values  Predicted Values
0       893.54            893.62
1        34.82             35.03
2       269.52            269.59
3       164.96            164.47
4       612.64            610.41
5       201.81            201.63
6       328.06            328.25
7       612.65            612.57
8       497.56            497.23
9       343.29            343.43


In [27]:
X_test_2, y_test_2 = data_0.drop('installment', axis=1), data_0.installment
""" X_test_2 = scaler.transform(X_test_2) """

y_test_pred_2 = model.predict(X_test_2)

results = pd.DataFrame({'True Values': y_test_2, 'Predicted Values': y_test_pred_2})
print(results.head(10))

C:\Users\edoardo.frapiccini\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but PolynomialFeatures was fitted without feature names
  warnings.warn(


    True Values  Predicted Values
4        609.33            265.31
26       284.45            164.55
36       115.52             70.00
48       411.01            183.89
54       244.36             73.57
69       117.14             69.24
72       271.27            170.13
81       356.48            215.70
82       285.21            171.71
84       267.47            164.63
